In [1]:
%run correlate_evolution.ipynb

%cd /work/clusterEvo/new_tests/eggNOG/

/work/clusterEvo/new_tests/eggNOG


In [2]:
import multiprocessing
import itertools
import pandas as pd
import plotly
from scipy.stats import pearsonr, mannwhitneyu
from scipy.spatial.distance import braycurtis
import igraph as ig
from Bio import SeqIO
import requests

In [3]:
single_copy = pd.read_parquet('single_copy_genes.parquet')
single_copy.head()

,group_id,num_proteins,num_taxa,members,taxa
1277,arCOG00079,112,112,"1041930.Mtc_1976,1054217.TALC_01539,1070774.J0...","[1041930, 1054217, 1070774, 1071085, 1094980, ..."
1279,arCOG00081,166,166,"1006006.Mcup_0318,1033806.HTIA_0777,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
1519,arCOG00354,112,112,"1033806.HTIA_1426,1041930.Mtc_0671,1042877.GQS...","[1033806, 1041930, 1042877, 1070774, 1071085, ..."
1520,arCOG00355,106,106,"1033806.HTIA_0952,1041930.Mtc_0275,1042877.GQS...","[1033806, 1041930, 1042877, 1070774, 1071085, ..."
1561,arCOG00410,168,168,"1006006.Mcup_0164,1033806.HTIA_1385,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."


In [6]:
taxa = set()
single_copy.taxa.iloc[0]

array([1041930, 1054217, 1070774, 1071085, 1094980, 1114856, 1121945,
       1131266, 1132509, 1197130, 1202768, 1210908, 1220534, 1227453,
       1227454, 1227457, 1227484, 1227487, 1227488, 1227495, 1227497,
       1227499, 1227500, 1230457, 1230460, 1236689, 1238425, 1261545,
       1293047, 1293048, 1324957, 1333523, 1448860, 1455608, 1457250,
       1459636,  192952,  222984,  224325,  224719,  243232,  263820,
        267377,  268739,  269797,  272569,  273075,  273116,   29540,
        304371,  309800,  323259,  339860,  348780,  351160,  358396,
        362976,  368407,  386456,  402880,  406327,  406552,  410358,
        413816,  416348,  419665,  420247,  426368,  436308,  439481,
        444158,  456320,  456442,  469382,  485914,  519442,  521011,
        523841,  523845,  543526,  547559,  565033,  572546,  573063,
        573064,  579137,  589924,  593750,  634497,  634498,  644281,
        647113,  662479,  673860,  679926,  693661,  694430,  694440,
        751944,  756

In [10]:
single_copy.query('num_taxa == 168')

,group_id,num_proteins,num_taxa,members,taxa
1561,arCOG00410,168,168,"1006006.Mcup_0164,1033806.HTIA_1385,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
1628,arCOG00487,168,168,"1006006.Mcup_0513,1033806.HTIA_1663,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
2087,arCOG01016,168,168,"1006006.Mcup_1994,1033806.HTIA_0370,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
2377,arCOG01344,168,168,"1006006.Mcup_0605,1033806.HTIA_1678,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
2563,arCOG01560,168,168,"1006006.Mcup_0030,1033806.HTIA_2422,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
2745,arCOG01751,168,168,"1006006.Mcup_0156,1033806.HTIA_0647,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
4590,arCOG04070,168,168,"1006006.Mcup_1967,1033806.HTIA_1952,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
4591,arCOG04071,168,168,"1006006.Mcup_1966,1033806.HTIA_1951,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
4592,arCOG04072,168,168,"1006006.Mcup_1965,1033806.HTIA_1950,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
4606,arCOG04086,168,168,"1006006.Mcup_1946,1033806.HTIA_1931,1041930.Mt...","[1006006, 1033806, 1041930, 1042877, 1054217, ..."
